In [1]:
import tulip as tlp
from tulip.interfaces import omega as omega_int
from tulip import transys, abstract, spec, synth
from visualization import graph_builder as gb
import networkx as nx
from tulip.transys import machines
from GR1_defaults import settings

from tulip import dumpsmach
import pickle

from omega.games import enumeration as enum

`omega.symbolic.symbolic` failed to import `dd.cudd`.
Will use `dd.autoref`.
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/mathset.py:719: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/mathset.py:719: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/labeled_graphs.py:1218: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/labeled_graphs.py:1226: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/labeled_graphs.py:1218: SyntaxWarning: "is" with a literal

In [2]:
path = 'Tgame/'

In [3]:
# System definition
sys = tlp.transys.FTS()

sys.atomic_propositions.add_from({'a0', 'a1', 'a2', 'a3'})
sys.states.add('c0', ap={'a0'})
sys.states.add('c1', ap={'a1'})
sys.states.add('c2', ap={'a2'})
sys.states.add('c3', ap={'a3'})
sys.states.initial.add('c0')    # start in state c0

sys.transitions.add_comb({'c0'}, {'c0', 'c1'})
# sys.transitions.add_comb({'c1'}, {'c1', 'c2'})
sys.transitions.add_comb({'c1'}, {'c0', 'c1', 'c2'})
# sys.transitions.add_comb({'c2'}, {'c2', 'c3'})
sys.transitions.add_comb({'c2'}, {'c1', 'c2', 'c3'})
# sys.transitions.add_comb({'c3'}, {'c3'})
sys.transitions.add_comb({'c3'}, {'c2', 'c3'})

In [4]:
# Variables
env_vars = {'b': (0, 4)}
sys_vars = {}

In [5]:
# Initialization
env_init = {'b = 0'}
sys_init = {}

In [6]:
# Safety
env_safe = {
    # Movement
    'b = 0 -> next(b) = 0 | next(b) = 1',
    'b = 1 -> next(b) = 1 | next(b) = 0 | next(b) = 2',
    'b = 2 -> next(b) = 2 | next(b) = 1 | next(b) = 3',
    'b = 3 -> next(b) = 3 | next(b) = 2 | next(b) = 4',
    'b = 4 -> next(b) = 4 | next(b) = 3',
}
sys_safe = {
    # No collision
    '!(a2 & b = 2)',
    # No being collided into
    # Commenting this out means that we are considering the game as having simultaneous updates
    '!(a2 & X(b = 2))'
}

In [7]:
# Progress
env_prog = {'b = 4'}
sys_prog = {'a3'}
    

specs = settings.set_specs(env_vars, sys_vars, env_init, sys_init,
                        env_safe, sys_safe, env_prog, sys_prog)
print(specs.pretty())

spec = tlp.synth._spec_plus_sys(specs, None, sys, False, False)
# Automaton class found in omega/omega/symbolic/temporal.py
aut = omega_int._grspec_to_automaton(spec)

ENVIRONMENT VARIABLES:
	b	(0, 4)

SYSTEM VARIABLES:
	(none)

FORMULA:
ASSUMPTION:
    INITIAL
	  (b = 0)
    SAFETY
	  [](b = 3 -> next(b) = 3 | next(b) = 2 | next(b) = 4)
	& [](b = 1 -> next(b) = 1 | next(b) = 0 | next(b) = 2)
	& [](b = 4 -> next(b) = 4 | next(b) = 3)
	& [](b = 0 -> next(b) = 0 | next(b) = 1)
	& [](b = 2 -> next(b) = 2 | next(b) = 1 | next(b) = 3)
    LIVENESS
	  []<>(b = 4)
GUARANTEE:
    SAFETY
	  [](!(a2 & b = 2))
	& [](!(a2 & X(b = 2)))
    LIVENESS
	  []<>(a3)



In [8]:
# Synthesize the controller
ctrl = tlp.synth.synthesize(specs, sys=sys)
assert ctrl is not None, 'unrealizable'
with open(path + "ctrl", "wb") as file:
    pickle.dump(ctrl, file)

dumpsmach.write_python_case(path + 'controller.py', ctrl, classname="sys_ctrl")

removed 0 nodes from 11 total


In [9]:
ctrl.plot()

/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/export/graph2dot.py:106: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/export/graph2dot.py:111: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/export/graph2dot.py:112: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/export/graph2dot.py:155: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/export/graph2dot.py:157: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech

ImportError: cannot import name 'make_str' from 'networkx.utils' (/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/networkx/utils/__init__.py)

In [113]:
# spec = tlp.synth._spec_plus_sys(specs, None, sys, False, False)
# # Automaton class found in omega/omega/symbolic/temporal.py
# aut = omega_int._grspec_to_automaton(specs)

In [114]:
# enum.action_to_steps(aut, env='env', sys='sys',qinit=aut.qinit)

In [115]:
# enum.action_to_steps(aut, env='env', sys='sys')

In [116]:
# enum.action_to_steps(aut, env='env', sys='impl')

In [10]:
# Graphing
filename = path + "graph"
attributes = ['color', 'shape']

# Making a graph of the asynchronous GR(1) game with deadends.
g0 = gb.game_graph(aut, env='env', sys='sys', remove_deadends=False, append_non_visited=False, qinit=aut.qinit)
h0 = gb._game_format_nx(g0, attributes)
pd0 = nx.drawing.nx_pydot.to_pydot(h0)
pd0.write_pdf(path + 'game.pdf')
with open(filename, "wb") as file:
    pickle.dump(g0, file)

about to create game graph
found winning set
before queue


/var/folders/ft/xrlc5_s96yq61k69b07g248r0000gn/T/ipykernel_3136/1215403325.py:8: DeprecationWarning: nx.nx_pydot.to_pydot depends on the pydot package, which hasknown issues and is not actively maintained.

See https://github.com/networkx/networkx/issues/5723
  pd0 = nx.drawing.nx_pydot.to_pydot(h0)


# Trying to satisfy the environment's liveness/progress conditions

## Find the nodes in the graph that satisfy any of the environment's progress conditions

In [118]:
# making environment progress conditions into a dictionary
env_prog_dict = {}
for elem in env_prog:
    split_lst = elem.split(' = ')
    key = split_lst[0]
    if len(split_lst) == 1:
        env_prog_dict[key] = True
    elif type(env_vars[key]) == list:
        value = split_lst[1]
        env_prog_dict[key] = env_vars[key].index(value[1:len(value)-1])
    else:
        env_prog_dict[key] = int(split_lst[1])


In [119]:
env_prog_dict

{'b': 4}

In [120]:
# find the nodes in the graph that satisfy any of the environment's progress conditions
# each entry in the dictionary is a list of nodes that satisfying each progress condition
env_prog_nodes = {}
for item in env_prog_dict.items():
    env_prog_nodes[item] = []
for node in g0.nodes:
    for item in env_prog_dict.items():
        if item in g0.nodes[node].items():
            env_prog_nodes[item].append(node)

In [156]:
env_prog_nodes

{('b', 4): [12, 13, 14, 15, 16, 19, 20, 23]}

In [121]:
for node in env_prog_nodes[('b', 4)]:
    print(g0.nodes[node])

{'b': 4, 'a0': True, 'a1': False, 'a3': False, 'a2': False, 'loc': 0, 'shape': 'box'}
{'b': 4, 'a0': False, 'a1': True, 'a3': False, 'a2': False, 'loc': 1, 'shape': 'oval'}
{'b': 4, 'a0': True, 'a1': False, 'a3': False, 'a2': False, 'loc': 0, 'shape': 'oval'}
{'b': 4, 'a0': False, 'a1': True, 'a3': False, 'a2': False, 'loc': 1, 'shape': 'box'}
{'b': 4, 'a0': False, 'a1': False, 'a3': False, 'a2': True, 'loc': 2, 'shape': 'oval'}
{'b': 4, 'a0': False, 'a1': False, 'a3': False, 'a2': True, 'loc': 2, 'shape': 'box'}
{'b': 4, 'a0': False, 'a1': False, 'a3': True, 'a2': False, 'loc': 3, 'shape': 'oval'}
{'b': 4, 'a0': False, 'a1': False, 'a3': True, 'a2': False, 'loc': 3, 'shape': 'box'}


In [122]:
env_prog_nodes

{('b', 4): [12, 13, 14, 15, 16, 19, 20, 23]}

## Find the cycles that contain nodes that satisfy each env progress condition

In [123]:
# find all the simple cycles in g0
cycles = list(nx.simple_cycles(g0))

In [124]:
cycles

[[0, 3, 5, 1],
 [0, 1],
 [3, 5],
 [3, 5, 6, 8],
 [6, 8, 9, 11],
 [6, 8],
 [9, 11],
 [9, 11, 12, 14],
 [12, 14],
 [33, 4],
 [33, 4, 31, 7],
 [33, 4, 35, 2],
 [7, 17, 10, 31],
 [7, 31],
 [17, 10],
 [17, 10, 15, 13],
 [13, 15],
 [2, 35],
 [36, 37],
 [20, 26, 22, 23],
 [20, 23],
 [22, 26],
 [22, 24, 25, 26],
 [24, 25, 27, 28],
 [24, 25],
 [27, 28],
 [27, 28, 29, 30],
 [29, 30]]

In [125]:
# make the cycles into frozensets (immutable sets)
old_cycles = set()
for cycle in cycles:
    old_cycles.add(frozenset(cycle))

num_iters = 0
while True:
    num_iters += 1
    new_cycles = old_cycles.copy()
    for cycle in old_cycles:
        for node in cycle:
            for cycle2 in old_cycles:
                if node in cycle2:
                    new_cycles.add(frozenset(cycle | cycle2))
    if new_cycles == old_cycles:
        print(num_iters)
        break
    else:
        old_cycles = new_cycles.copy()

    

3


In [126]:
print("num_iters", num_iters)
print("len(cycles)", len(cycles))
print("len(new_cycles)", len(new_cycles))

num_iters 3
len(cycles) 28
len(new_cycles) 46


In [127]:
# to make the code work without having to change everything
cycles = new_cycles

In [128]:
env_prog_nodes

{('b', 4): [12, 13, 14, 15, 16, 19, 20, 23]}

In [129]:
# Filter cycles that contain nodes that satisfy each type of progress condition
prog_satisfying_cycles = []
for cycle in cycles:
    satisfies_prog_counter = 0
    for lst in env_prog_nodes.values():
        for node in lst:
            if node in cycle:
                satisfies_prog_counter += 1
                break
    if satisfies_prog_counter == len(env_prog_dict.values()):
        prog_satisfying_cycles.append(cycle)

In [130]:
prog_satisfying_cycles

[frozenset({20, 22, 23, 24, 25, 26, 27, 28, 29, 30}),
 frozenset({7, 10, 13, 15, 17, 31}),
 frozenset({0, 1, 3, 5, 6, 8, 9, 11, 12, 14}),
 frozenset({2, 4, 7, 10, 13, 15, 17, 31, 33, 35}),
 frozenset({20, 22, 23, 24, 25, 26}),
 frozenset({13, 15}),
 frozenset({6, 8, 9, 11, 12, 14}),
 frozenset({20, 22, 23, 24, 25, 26, 27, 28}),
 frozenset({9, 11, 12, 14}),
 frozenset({12, 14}),
 frozenset({4, 7, 10, 13, 15, 17, 31, 33}),
 frozenset({3, 5, 6, 8, 9, 11, 12, 14}),
 frozenset({20, 22, 23, 26}),
 frozenset({10, 13, 15, 17}),
 frozenset({20, 23})]

## See if any of these cycles can have the system leave the cycle (with no return)

In [131]:
g0.nodes[0]

{'b': 0,
 'a0': True,
 'a1': False,
 'a3': False,
 'a2': False,
 'loc': 0,
 'shape': 'oval'}

In [132]:
# see if any of these cycles can have the system leave the cycle (regardless of if it can return)
leave_counter = 0
num_cycles_left = 0
for cycle in prog_satisfying_cycles:
    cycle_left = False
    for node in cycle:
        # if g0.nodes[node]['shape'] != 'oval': 
        if g0.nodes[node]['shape'] == 'oval': 
            continue
        # node is a node from which the system takes the action
        for succ in g0.successors(node):
            if succ not in cycle:
                leave_counter += 1
                cycle_left = True
    if cycle_left:
        num_cycles_left += 1
print("leave_counter", leave_counter)
print("num_cycles_left", num_cycles_left)
print('num_cycles', len(prog_satisfying_cycles))

leave_counter 30
num_cycles_left 10
num_cycles 15


## Remove any cycles with the system leaving

In [133]:
# This code is very similar to the code above. Fix this later
permanent_prog_cycles = []
for cycle in prog_satisfying_cycles:
    cycle_left = False
    for node in cycle:
        # if g0.nodes[node]['shape'] != 'oval':
        if g0.nodes[node]['shape'] == 'oval':
            continue
        # node is a node from which the system takes the action
        for succ in g0.successors(node):
            if succ not in cycle:
                leave_counter += 1
                cycle_left = True
    if cycle_left:
        num_cycles_left += 1
    else:
        permanent_prog_cycles.append(cycle)

In [134]:
permanent_prog_cycles

[frozenset({20, 22, 23, 24, 25, 26, 27, 28, 29, 30}),
 frozenset({20, 22, 23, 24, 25, 26}),
 frozenset({20, 22, 23, 24, 25, 26, 27, 28}),
 frozenset({20, 22, 23, 26}),
 frozenset({20, 23})]

In [162]:
largest_cycles = []
for cycle in permanent_prog_cycles:
    is_subset = False
    for other_cycle in permanent_prog_cycles:
        if cycle != other_cycle and cycle.issubset(other_cycle):
            is_subset = True
            break
    if not is_subset:
        largest_cycles.append(cycle)

In [163]:
largest_cycles

[frozenset({20, 22, 23, 24, 25, 26, 27, 28, 29, 30})]

Do we acccount for the case where you enter one cycle then go into another via a path?
I do not think we do. 

Do we account for the case where multiple system actions can keep us within some cycle?
I also think no.

We only find cycles in which the system action cannot leave the cycle.

Need to fix.

## (Not necessary) Find all paths from the initial node to the cycles

In [37]:
# def is_prefix(list1, list2):
#     # Compare the elements of both lists up to the length of the shorter list
#     min_length = min(len(list1), len(list2))
#     for i in range(min_length):
#         if list1[i] != list2[i]:
#             return False
#     return True

In [38]:
# Find all the paths from the initial node(s) to the cycles satisfying the progress conditions
# ASSUMPTION: g0.nodes[0] is the ONLY initial node
# paths_to_cycle = []
# for cycle in prog_satisfying_cycles:
#     for node in cycle:
#         new_paths = list(nx.all_simple_paths(g0, 0, node))
#         # I do not want to have 2 paths, one of which is a subset of the other. 
#         # I just want the subset.
#         for new_path in new_paths[:]:
#             for existing_path in paths_to_cycle[:]:
#                 if is_prefix(new_path, existing_path):
#                     if len(new_path) < len(existing_path):
#                         paths_to_cycle.remove(existing_path)
#                     else:
#                         new_paths.remove(new_path)
#                         break
#         paths_to_cycle += new_paths

new problem: This does not take into account that the system may take some action 
that will take us away from one of these paths. Do we have to care?
Yes. We want to always satisfy the liveness conditions, no matter what the system does.

Should I do some winning set calculations? I can't just use the winning set 
functions because I want to use the GR1 graph that contains the actions the 
system can take that satisfy its winning condition. 

Okay let's say that there is a safe action that the system can take but will lead to
the environment taking the system to a red node. Is that in our graph? No... it does not seem so.
Wouldn't we want a hard test to find this case? Do we just assume that the system has failed at this point?

okay so should we just assume the graph is ideal
and then figure out the paths that allow the environment to satisfy its progress conditions,
regardless of what the system does

In [39]:
# len(paths_to_cycle)

17

In [40]:
# paths_to_cycle

[[0, 3, 4, 33, 34, 35],
 [0, 1, 2, 11, 5, 8, 10, 13],
 [0, 3, 4, 1, 2, 11, 5, 8, 10, 13],
 [0, 3, 4, 33, 10, 13],
 [0, 3, 5, 8, 10, 13],
 [0, 1, 2, 6, 7, 31, 30, 27, 28, 24, 25, 26],
 [0, 1, 2, 6, 7, 32, 28, 24, 25, 26],
 [0, 1, 2, 11, 5, 6, 7, 31, 30, 27, 28, 24, 25, 26],
 [0, 1, 2, 11, 5, 6, 7, 32, 28, 24, 25, 26],
 [0, 3, 4, 1, 2, 6, 7, 31, 30, 27, 28, 24, 25, 26],
 [0, 3, 4, 1, 2, 6, 7, 32, 28, 24, 25, 26],
 [0, 3, 4, 1, 2, 11, 5, 6, 7, 31, 30, 27, 28, 24, 25, 26],
 [0, 3, 4, 1, 2, 11, 5, 6, 7, 32, 28, 24, 25, 26],
 [0, 3, 4, 33, 10, 11, 5, 6, 7, 31, 30, 27, 28, 24, 25, 26],
 [0, 3, 4, 33, 10, 11, 5, 6, 7, 32, 28, 24, 25, 26],
 [0, 3, 5, 6, 7, 31, 30, 27, 28, 24, 25, 26],
 [0, 3, 5, 6, 7, 32, 28, 24, 25, 26]]

Should I make a graph that adds a transition from one env node to another only if all system actions from 
the env node go to the other

Then, I do the same code on this

Wait, I should do it on sets. Only if all system actions from the env node go to a winning set lol


## Find the "levels" that lead to the environment progress cycles

In [164]:
# getting the nodes in the cycles that satisfy the env progress conditions
nodes_satisfying_prog = set()
# for lst in prog_satisfying_cycles:
for lst in largest_cycles:
    nodes_satisfying_prog |= set(lst)

In [165]:
nodes_satisfying_prog

{20, 22, 23, 24, 25, 26, 27, 28, 29, 30}

In [147]:
# calculate the "levels" from the env prog cycle outwards
R = {}
i = 0
R[i] = nodes_satisfying_prog
while True:
    new_nodes = set()
    for node in R[i]:
        new_nodes |= set(g0.predecessors(node))
    R[i+1] = new_nodes | R[i]

    new_nodes = set()
    for node in R[i+1]:
        for prev in set(g0.predecessors(node)):
            all_transitions_to_Ri1 = True
            for trans in set(g0.successors(prev)):
                if trans not in R[i+1]:
                    all_transitions_to_Ri1 = False
                    break
            if all_transitions_to_Ri1:
                new_nodes.add(prev)
    R[i+2] = new_nodes | R[i+1]

    if R[i+2] == R[i]:
        R.pop(i+2)
        R.pop(i+1)
        break
    else:
        i += 2

In [166]:
# the initial state is in the largest "level" out of the env prog cycle
# i.e. there is a path for the environment to the progress satisfying cycles,
# regardless of the moves the system takes
# ASSUMPTION the initial state is ONLY node 0
0 in R[len(R)-1]

True

# Start from initial state(s?) and go to the union of cycles

In [ ]:
def go_to_cycles(robustness, R, g):
    

In [3]:
??nx.simple_cycles

Signature: nx.simple_cycles(G, length_bound=None)
Source:   
def simple_cycles(G, length_bound=None):
    """Find simple cycles (elementary circuits) of a graph.

    A `simple cycle`, or `elementary circuit`, is a closed path where
    no node appears twice.  In a directed graph, two simple cycles are distinct
    if they are not cyclic permutations of each other.  In an undirected graph,
    two simple cycles are distinct if they are not cyclic permutations of each
    other nor of the other's reversal.

    Optionally, the cycles are bounded in length.  In the unbounded case, we use
    a nonrecursive, iterator/generator version of Johnson's algorithm [1]_.  In
    the bounded case, we use a version of the algorithm of Gupta and
    Suzumura[2]_. There may be better algorithms for some cases [3]_ [4]_ [5]_.

    The algorithms of Johnson, and Gupta and Suzumura, are enhanced by some
    well-known preprocessing techniques.  When G is directed, we restrict our
    attention to strong

In [2]:
nx.__version__

'3.1'

In [7]:
import sys
!{sys.executable} -m pip install --upgrade networkx

In [6]:
!{sys.executable} -m pip install --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/8a/6a/19e9fe04fca059ccf770861c7d5721ab4c2aebc539889e97c7977528a53b/pip-24.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.2
    Uninstalling pip-23.2:
      Successfully uninstalled pip-23.2


In [4]:
import sys
print(sys.version)

3.8.16 (default, Mar  1 2023, 21:19:10) 
[Clang 14.0.6 ]


In [1]:
{1,2,3}.issubset({1,2})

False

In [2]:
{1,2}.issubset({1,2,3})

True

In [3]:
{1,2,3}.issubset({1,2,3})

True

In [15]:
a = {frozenset({1,2}),frozenset({4,2})}
b = a.copy()

In [16]:
b.remove(frozenset({1,2}))

In [17]:
b

{frozenset({2, 4})}

In [18]:
b.remove(frozenset({1,2}))

KeyError: frozenset({1, 2})

In [21]:
{1,2,3}.intersection({4,2,3})

{2, 3}